In [1]:
import os
import numpy as np
import pandas as pd
import astropy.units as u

from astropy.table import Table
from astropy.io import ascii
from cdspyreadme import CDSTablesMaker
from glob import glob

tablemaker = CDSTablesMaker()

## Standardisation

In [2]:
norm = pd.read_csv('../data/networks/network_10/network_10_norm.csv')
norm

,column,norm_column,loc,scale
0,frac_evol,frac_evol_norm,0.865,0.651
1,initial_mass,initial_mass_norm,1.000,0.118
2,initial_MLT,initial_MLT_norm,1.900,0.338
3,initial_Yinit,initial_Yinit_norm,0.280,0.028
4,initial_Zinit,initial_Zinit_norm,0.017,0.011
5,log_star_age,log_star_age_norm,0.790,0.467
6,effective_T,effective_T_norm,5566.772,601.172
7,radius,radius_norm,1.224,0.503
8,delta_nu_fit,delta_nu_fit_norm,100.720,42.582
9,star_feh,star_feh_norm,0.081,0.361


In [3]:
norm['type'] = [r'\textbf{Input}']*5 + [r'\textbf{Output}']*5
norm['name'] = [
    r'$f_\mathrm{evol}$', 
    r'$M\,(\si{\solarmass})$', 
    r'$\mlt$', 
    r'$Y_\mathrm{init}$', 
    r'$Z_\mathrm{init}$',
    r'$\log(\tau/\si{\giga\year})$', 
    r'$\teff\,(\si{\kelvin})$', 
    r'$R\,(\si{\solarradius})$',
    r'$\dnu\,(\si{\micro\hertz})$',
    r'$\metallicity_\mathrm{surf}\,(\si{\dex})$',
]

In [4]:
columns = [
    norm['type'].to_numpy(),
    norm['name'].to_numpy(),
]

In [5]:
norm[['loc', 'scale']].T

,0,1,2,3,4,5,6,7,8,9
loc,0.865,1.000,1.900,0.280,0.017,0.790,5566.772,1.224,100.720,0.081
scale,0.651,0.118,0.338,0.028,0.011,0.467,601.172,0.503,42.582,0.361


In [6]:
table = pd.DataFrame(norm[['loc', 'scale']].T.to_numpy(), index=[r'$\mu_{1/2}$', '$\sigma$'], columns=columns)
table

\textbf{Input}                                                  \
            $f_\mathrm{evol}$ $M\,(\si{\solarmass})$ $\mlt$ $Y_\mathrm{init}$   
$\mu_{1/2}$             0.865                  1.000  1.900             0.280   
$\sigma$                0.651                  0.118  0.338             0.028   

                                           \textbf{Output}  \
            $Z_\mathrm{init}$ $\log(\tau/\si{\giga\year})$   
$\mu_{1/2}$             0.017                        0.790   
$\sigma$                0.011                        0.467   

                                                              \
            $\teff\,(\si{\kelvin})$ $R\,(\si{\solarradius})$   
$\mu_{1/2}$                5566.772                    1.224   
$\sigma$                    601.172                    0.503   

                                         \
            $\dnu\,(\si{\micro\hertz})$   
$\mu_{1/2}$                     100.720   
$\sigma$                         42.582   

                                                       
            $\metallicity_\mathrm{surf}\,(\si{\dex})$  
$\mu_{1/2}$                                     0.081  
$\sigma$                                        0.361

In [7]:
with open('../paper/standardisation.tex', 'w') as file:
    s = table.to_latex(na_rep='---', escape=False, 
                          column_format='c|'+'c'*len(table[r'\textbf{Input}'].columns)+'|'+\
                                        'c'*len(table[r'\textbf{Input}'].columns),
                          multicolumn_format='c|',
                         )
    s = s.replace(r'\multicolumn{5}{c|}{\textbf{Output}}', r'\multicolumn{5}{c}{\textbf{Output}}')
    file.write(s)

## Hyperparameter Results

In [17]:
pop_paths = glob('../modelling/final_models/stars_results/*/analysis/results.csv')

In [18]:
pop_paths += glob('../modelling/final_models/population*/*/*/*/results.csv')
pop_paths

['../modelling/final_models/stars_results/DR14_ASPC/analysis/results.csv',
 '../modelling/final_models/population+sun_results/max_pool/DR14_ASPC/population/results.csv',
 '../modelling/final_models/population+sun_results/partial_pool/DR14_ASPC/population/results.csv',
 '../modelling/final_models/population_results/max_pool/DR14_ASPC/population/results.csv',
 '../modelling/final_models/population_results/partial_pool/DR14_ASPC/population/results.csv']

In [33]:
names = ['NP', 'MPS', 'PPS', 'MP', 'PP']
results = {}
for name, path in zip(names, pop_paths):
    results[name] = pd.read_csv(path)

results = pd.concat(results, names=['model', 'index'])

In [34]:
results = results.reset_index().drop(columns='index')
results

,model,dydz_mean,dydz_sd,dydz_hdi_3%,dydz_hdi_97%,dydz_mcse_mean,dydz_mcse_sd,dydz_ess_mean,dydz_ess_sd,dydz_ess_bulk,...,mlt_hpd_97%,mlt_mcse_mean,mlt_mcse_sd,mlt_mean,mlt_r_hat,mlt_sd,mu_mlt_hpd_3%,mu_mlt_hpd_97%,sigma_mlt_hpd_3%,sigma_mlt_hpd_97%
0,NP,1.693032,0.207203,1.300726,2.082069,0.001329,0.000945,24303.929688,24042.346859,24306.782658,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MPS,0.751483,0.259093,NaN,NaN,0.007738,0.005473,1121.166575,1121.166575,1115.487716,...,2.144985,0.000309,0.000218,2.089183,1.000708,0.029869,NaN,NaN,NaN,NaN
2,PPS,1.064252,0.285046,NaN,NaN,0.009209,0.006592,958.166606,935.412058,1000.421792,...,NaN,NaN,NaN,NaN,NaN,NaN,1.738124,2.077312,0.036031,0.244246
3,MP,1.617958,0.437875,NaN,NaN,0.016299,0.011530,721.703020,721.703020,722.095299,...,1.876503,0.002845,0.002013,1.733456,1.017069,0.073501,NaN,NaN,NaN,NaN
4,PP,1.610308,0.439977,NaN,NaN,0.015518,0.010977,803.884882,803.884882,804.112649,...,NaN,NaN,NaN,NaN,NaN,NaN,1.605563,1.898497,0.005282,0.146947


In [35]:
results = results.set_index('model').loc[['NP', 'PP', 'PPS', 'MP', 'MPS']]
results = results.reset_index()

In [36]:
def make_result(name, fmt='.2f'):
    res = results.loc[results[f'{name}_50th'].notna()]
    func = lambda x: ('{' + f':{fmt}' + '}').format(x)
    upper = (res[f'{name}_84th'] - res[f'{name}_50th']).apply(func)
    lower = (res[f'{name}_50th'] - res[f'{name}_16th']).apply(func)
    return r'$' + res[f'{name}_50th'].apply(func) + r'\substack{+' + \
            upper + r' \\ -' + lower + r'}$'

In [37]:
results[r'\textbf{Model}'] = results['model']
results[r'$\Delta Y/\Delta Z$'] = make_result('dydz', fmt='.2f')
# results[r'$Y_P$'] = make_result('yp', fmt='.3f')
results[r'$\sigma_Y$'] = make_result('sigma_yi', fmt='.4f')
results[r'$\mu_\alpha$'] = make_result('mu_mlt', fmt='.3f')
results[r'$\sigma_\alpha$'] = make_result('sigma_mlt', fmt='.3f')
results[r'$\alpha_\mathrm{mlt}$'] = make_result('mlt', fmt='.3f')


In [38]:
table = results.iloc[:, -6:]
table

,\textbf{Model},$\Delta Y/\Delta Z$,$\sigma_Y$,$\mu_\alpha$,$\sigma_\alpha$,$\alpha_\mathrm{mlt}$
0,NP,$1.69\substack{+0.21 \\ -0.21}$,$0.0074\substack{+0.0026 \\ -0.0022}$,$1.954\substack{+0.040 \\ -0.041}$,$0.065\substack{+0.030 \\ -0.024}$,NaN
1,PP,$1.60\substack{+0.45 \\ -0.42}$,$0.0051\substack{+0.0045 \\ -0.0027}$,$1.742\substack{+0.081 \\ -0.070}$,$0.056\substack{+0.051 \\ -0.030}$,NaN
2,PPS,$1.05\substack{+0.28 \\ -0.25}$,$0.0045\substack{+0.0038 \\ -0.0023}$,$1.900\substack{+0.095 \\ -0.088}$,$0.133\substack{+0.057 \\ -0.047}$,NaN
3,MP,$1.60\substack{+0.45 \\ -0.42}$,$0.0051\substack{+0.0044 \\ -0.0027}$,NaN,NaN,$1.728\substack{+0.077 \\ -0.066}$
4,MPS,$0.76\substack{+0.24 \\ -0.27}$,$0.0049\substack{+0.0039 \\ -0.0025}$,NaN,NaN,$2.088\substack{+0.031 \\ -0.029}$


In [39]:
with open('../paper/hyperparam_results.tex', 'w') as file:
    file.write(table.to_latex(na_rep='---', escape=False, 
                              index=False, column_format='c'*len(table.columns)))

## Stars inputs

In [6]:
def make_value(results, name, fmt='.2f'):
    res = results.loc[results[name].notna()]
    func = lambda x: ('{' + f':{fmt}' + '}').format(x)
    return r'$' + res[name].apply(func) + r'\pm' + \
            res[f'{name}_err'].apply(func) + r'$'

In [7]:
scale = 'DR14_ASPC'
stars_in = pd.read_csv(f'../data/stellr/inputs_{scale}.csv')
stars_in.head()

,name,teff,teff_err,lum,lum_err,dnu,dnu_err,mhs,mhs_err,logg,logg_err,bad_data,on_grid_1e,on_grid_2e
0,KIC5950854,5914.0347,134.60000,1.813844,0.062565,96.857,0.194,-0.173085,0.073964,4.237158,0.009836,0,1,1
1,KIC7799349,4976.7870,82.60566,4.421854,0.138701,33.299,0.322,0.284619,0.029484,3.670984,0.007718,0,0,0
2,KIC5822889,5184.0103,94.43386,12.830989,0.707319,19.708,0.127,-0.026246,0.039186,3.401706,0.012167,0,0,0
3,KIC12069569,6062.4185,168.10576,7.144911,0.291244,44.687,2.162,-0.084244,0.066659,3.825712,0.060999,0,0,0
4,KIC6947945,5334.4380,103.46670,3.806103,0.153818,43.660,1.416,-0.041512,0.043107,3.794063,0.011004,0,0,0


In [8]:
stars_in[(stars_in['bad_data']==0) & (stars_in['on_grid_1e']==1)]

,name,teff,teff_err,lum,lum_err,dnu,dnu_err,mhs,mhs_err,logg,logg_err,bad_data,on_grid_1e,on_grid_2e
0,KIC5950854,5914.0347,134.600000,1.813844,0.062565,96.857,0.194,-0.173085,0.073964,4.237158,0.009836,0,1,1
8,KIC3942719,5653.2954,126.502540,4.631670,0.202205,45.463,1.208,-0.314923,0.060012,3.838612,0.012941,0,1,0
10,KIC4457351,5266.6953,100.036446,2.538945,0.095094,61.016,2.546,-0.279843,0.047070,3.964330,0.017045,0,1,0
15,KIC3223000,5979.4927,136.427060,1.932630,0.086986,110.919,1.662,-0.152958,0.075412,4.365700,0.095282,0,1,1
22,KIC7429287,5762.0845,133.098820,2.383776,0.079896,71.244,1.211,-0.305247,0.070656,4.076474,0.016897,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,KIC9143749,5735.3600,124.107180,3.361808,0.121795,64.977,1.891,-0.010952,0.071011,4.027943,0.022508,0,1,0
384,KIC8424992,5517.5083,128.721820,0.985946,0.029900,120.914,0.305,-0.095701,0.074718,4.340031,0.014748,0,1,1
387,KIC9266926,5781.6396,133.608100,2.231382,0.076240,75.081,1.458,-0.298283,0.071068,4.070394,0.017422,0,1,1
391,KIC3735871,5960.3850,131.760040,1.366482,0.043845,122.856,0.472,-0.029106,0.075043,4.404094,0.012215,0,1,0


In [9]:
df = stars_in[(stars_in['bad_data']==0) & (stars_in['on_grid_1e']==1)].iloc[:, :-5].sort_values('name')
table = df.head(5).copy()

In [10]:
table[r'\textbf{Name}'] = table['name']
table[r'$\teff\,(\si{\kelvin})$'] = make_value(table, 'teff', '.0f')
table[r'$L\,(\si{\solarluminosity})$'] = make_value(table, 'lum', '.3f')
table[r'$\dnu\,(\si{\micro\hertz})$'] = make_value(table, 'dnu') 
table[r'$\metallicity_\mathrm{surf}\,(\si{\dex})$'] = make_value(table, 'mhs', '.3f')
table.head()

,name,teff,teff_err,lum,lum_err,dnu,dnu_err,mhs,mhs_err,\textbf{Name},"$\teff\,(\si{\kelvin})$","$L\,(\si{\solarluminosity})$","$\dnu\,(\si{\micro\hertz})$","$\metallicity_\mathrm{surf}\,(\si{\dex})$"
295,KIC10079226,5928.8413,124.836624,1.571074,0.048691,116.036,0.735,0.158945,0.074398,KIC10079226,$5929\pm125$,$1.571\pm0.049$,$116.04\pm0.73$,$0.159\pm0.074$
193,KIC10215584,5666.9160,119.333700,1.636568,0.062670,115.158,2.834,0.043014,0.068864,KIC10215584,$5667\pm119$,$1.637\pm0.063$,$115.16\pm2.83$,$0.043\pm0.069$
224,KIC10319352,5456.1690,106.654010,1.848181,0.056483,78.746,1.732,0.265175,0.064762,KIC10319352,$5456\pm107$,$1.848\pm0.056$,$78.75\pm1.73$,$0.265\pm0.065$
297,KIC10322381,6146.7905,148.578700,2.444552,0.079126,86.644,6.572,-0.316814,0.079113,KIC10322381,$6147\pm149$,$2.445\pm0.079$,$86.64\pm6.57$,$-0.317\pm0.079$
273,KIC10417911,5628.2570,109.985430,3.408157,0.115068,56.138,2.099,0.335723,0.068119,KIC10417911,$5628\pm110$,$3.408\pm0.115$,$56.14\pm2.10$,$0.336\pm0.068$


In [11]:
header = [
    r'\textbf{Name}',
    r'$\teff\,(\si{\kelvin})$',
    r'$\sigma_{\teff}\,(\si{\kelvin})$',
    r'$L\,(\si{\solarluminosity})$',
    r'$\sigma_L\,(\si{\solarluminosity})$',
    r'$\dnu\,(\si{\micro\hertz})$',
    r'$\sigma_{\dnu}\,(\si{\micro\hertz})$',
    r'$\metallicity_\mathrm{surf}\,(\si{\dex})$',
    r'$\sigma_{\metallicity}\,(\si{\dex})$',
#     r'$\log{g}\,(\si{\dex})$',
#     r'$\sigma_{\log{g}}\,(\si{\dex})$'
]

In [12]:
with open('../paper/tables/stars_inputs.tex', 'w') as file:
    file.write(table.iloc[:, -5:].to_latex(na_rep='---', escape=False,
#                               header=header, 
#                               float_format="{:0.2f}".format,
                              index=False, column_format='ccccc'))

In [13]:
ap_table = Table.from_pandas(df)

In [14]:
ap_table['name'].description = 'Star identifier'

In [15]:
ap_table['teff'].unit = u.K
ap_table['teff_err'].unit = u.K

In [16]:
ap_table['teff'].description = "Effective temperature"
ap_table['teff_err'].description = "uncertainty"

In [17]:
ap_table['lum'].unit = u.solLum
ap_table['lum_err'].unit = u.solLum

In [18]:
ap_table['lum'].description = "Luminosity"
ap_table['lum_err'].description = "uncertainty"

In [19]:
ap_table['dnu'].unit = u.microhertz
ap_table['dnu_err'].unit = u.microhertz

In [20]:
ap_table['dnu'].description = "Large frequency separation"
ap_table['dnu_err'].description = "uncertainty"

In [21]:
ap_table['mhs'].unit = u.dex
ap_table['mhs_err'].unit = u.dex

In [22]:
ap_table['mhs'].description = "Surface metallicity"
ap_table['mhs_err'].description = "uncertainty"

In [23]:
tablemaker.addTable(ap_table, name="table1", 
                    description='The observables and their respective uncertainties for the sample of 81 stars.')

In [38]:
# with open('stars_inputs.txt', 'w') as file:
#     ascii.write(ap_table, output=file, format='ipac')

## Stars Results

In [2]:
def make_result(results, name, fmt='.2f'):
    res = results.loc[results[f'{name}_50th'].notna()]
    func = lambda x: ('{' + f':{fmt}' + '}').format(x)
    upper = (res[f'{name}_84th'] - res[f'{name}_50th']).apply(func)
    lower = (res[f'{name}_50th'] - res[f'{name}_16th']).apply(func)
    return r'$' + res[f'{name}_50th'].apply(func) + r'\substack{+' + \
            upper + r' \\ -' + lower + r'}$'

In [3]:
scale = 'DR14_ASPC'
star_paths = glob('../modelling/final_models/population*/*/*/*/*/results.csv')
star_paths

['../modelling/final_models/population+sun_results/max_pool/DR14_ASPC/population/stars/results.csv',
 '../modelling/final_models/population+sun_results/partial_pool/DR14_ASPC/population/stars/results.csv',
 '../modelling/final_models/population_results/max_pool/DR14_ASPC/population/stars/results.csv',
 '../modelling/final_models/population_results/partial_pool/DR14_ASPC/population/stars/results.csv']

In [24]:
star_paths_dict = {
    'PP': star_paths[3],
    'PPS': star_paths[1],
    'MP': star_paths[2],
    'MPS': star_paths[0],
}

In [25]:
stars_out = {}

stars_out['NP'] = pd.read_csv('../modelling/final_models/stars_results/DR14_ASPC/stars/results.csv').sort_values('name')

In [26]:
for key, path in star_paths_dict.items():
    stars_out[key] = pd.read_csv(path).sort_values('name')

In [27]:
tables = {}

for key, val in stars_out.items():
    tables[key] = pd.DataFrame()
    tables[key][r'\textbf{Name}'] = val['name']
    tables[key][r'$f_\mathrm{evol}$'] = make_result(val, 'f_evol')
    tables[key][r'$M\,(\si{\solarmass})$'] = make_result(val, 'mass')
    if 'mlt_50th' in val.columns:
        tables[key][r'$\mlt$'] = make_result(val, 'mlt')
    tables[key][r'$Y_\mathrm{init}$'] = make_result(val, 'yi') 
    tables[key][r'$Z_\mathrm{init}$'] = make_result(val, 'zi', '.3f')
#     tables[key][r'$\metallicity_\mathrm{init}\,(\si{\dex})$'] = make_result(val, 'mhi')    
    tables[key][r'$\tau\,(\si{\giga\year})$'] = make_result(val, 'age', '.1f')
    tables[key][r'$\teff\,(\si{\kelvin})$'] = make_result(val, 'teff', '.0f')
    tables[key][r'$R\,(\si{\solarradius})$'] = make_result(val, 'rad')
    tables[key][r'$\dnu\,(\si{\micro\hertz})$'] = make_result(val, 'dnu', '.1f')
    tables[key][r'$\metallicity_\mathrm{surf}\,(\si{\dex})$'] = make_result(val, 'mhs')

In [28]:
for key, table in tables.items():
    with open(f'../paper/tables/stars_outputs_{key}.tex', 'w') as file:
        file.write(table.head(5).to_latex(na_rep='---', escape=False,
                                  index=False, column_format='c'+'r'*(len(table.columns)-1)))

In [29]:
ap_tables = {}

for key, val in stars_out.items():
    ap_tables[key] = Table.from_pandas(val)

In [30]:
descriptions = {
    'f_evol': 'Fractional evolutionary phase (0-1 ~ core hydrogen burning, 1-2 ~ shell hydrogen burning)',
    'mass': 'Mass',
    'mlt': 'Mixing-length theory parameter',
    'yi': 'Initial helium fraction',
    'zi': 'Initial heavy-elements fraction',
    'mhi': 'Initial metallicity',
    'age': 'Age',
    'teff': 'Effective temperature',
    'rad': 'Radius',
    'lum': 'Luminosity',
    'dnu': 'Large frequency separation',
    'mhs': 'Surface metallicity'
}

In [31]:
def make_ap_result(old, new, name, unit=1):
    new[name] = old[f'{name}_50th'] * unit
    new[name].description = descriptions[name]
    pe = f'+e_{name}'
    new[pe] = (old[f'{name}_84th'] - old[f'{name}_50th']) * unit
    new[pe].description = 'upper uncertainty'
    
    me = f'-e_{name}'
    new[me] = (old[f'{name}_50th'] - old[f'{name}_16th']) * unit
    new[me].description = 'lower uncertainty'

In [32]:
out_tables = {}

In [33]:
i = 5

for key, val in ap_tables.items():
    out_tables[key] = Table()
    out_tables[key]['name'] = ap_tables[key]['name']
    out_tables[key]['name'].description = 'Star identifier'
    make_ap_result(ap_tables[key], out_tables[key], 'f_evol')
    make_ap_result(ap_tables[key], out_tables[key], 'mass', u.solMass)
    if 'mlt_50th' in val.columns:
        make_ap_result(ap_tables[key], out_tables[key], 'mlt')
    make_ap_result(ap_tables[key], out_tables[key], 'yi')
    make_ap_result(ap_tables[key], out_tables[key], 'zi')
    make_ap_result(ap_tables[key], out_tables[key], 'mhi', u.dex)
    make_ap_result(ap_tables[key], out_tables[key], 'age', u.gigayear)
    make_ap_result(ap_tables[key], out_tables[key], 'teff', u.K)
    make_ap_result(ap_tables[key], out_tables[key], 'rad', u.solRad)
    make_ap_result(ap_tables[key], out_tables[key], 'lum', u.solLum)
    make_ap_result(ap_tables[key], out_tables[key], 'dnu', u.microhertz)
    make_ap_result(ap_tables[key], out_tables[key], 'mhs', u.dex)
    if i == 5:
        desc = 'The median of the marginalised posterior samples for each parameter output by the NP model, ' + \
            'with their respective upper and lower 68 per cent credible intervals.'
    else:
        desc = f'The same as Table 5 but for the {key} model.'
    tablemaker.addTable(out_tables[key], name=f"table{i}", 
                        description=desc)
    i += 1

In [160]:
# for key, val in out_tables.items():
#     print(key)
#     with open(f'stars_outputs_{key}.txt', 'w') as file:
#         ascii.write(val, output=file, format='ipac')

## Solar data

In [18]:
sun = pd.DataFrame({
    'name': 'SUN',
    'mass': 1.0, 'mass_err': 0.001,
    'age': 4.6, 'age_err': 0.1,
    'teff': 5777., 'teff_err': 20.,
    'rad': 1.0, 'rad_err': 0.001,
    'lum': 1.0, 'lum_err': 0.01,
    'dnu': 135.1, 'dnu_err': 0.2,
    'mhs': 0.0, 'mhs_err': 0.01,
}, index=[0])
sun

,name,mass,mass_err,age,age_err,teff,teff_err,rad,rad_err,lum,lum_err,dnu,dnu_err,mhs,mhs_err
0,SUN,1.0,0.001,4.6,0.1,5777.0,20.0,1.0,0.001,1.0,0.01,135.1,0.2,0.0,0.01


In [20]:
# cols = [r'$\mu$', r'$\sigma$']
cols = ['Value', 'Reference']
sun_in = pd.DataFrame(columns=cols, index=pd.Index([], name=r'\textbf{Input}'))

In [22]:
references = [
    '---',
    r'\citet{Connelly.Bizzarro.ea2012}',
    r'\citet{Scott.Grevesse.ea2015}',
    r'---',
    r'---',
    r'\citet{Huber.Bedding.ea2011}',
    r'\citet{Asplund.Grevesse.ea2009}',
]

In [36]:
sun_in.loc[r'$M\,(\si{\solarmass})$', 'Value'] = make_value(sun, 'mass', '.3f').loc[0]
sun_in.loc[r'$\tau\,(\si{\giga\year})$', 'Value'] = make_value(sun, 'age', '.1f').loc[0]
sun_in.loc[r'$\teff\,(\si{\kelvin})$', 'Value'] = make_value(sun, 'teff', '.0f').loc[0]
sun_in.loc[r'$R\,(\si{\solarradius})$', 'Value'] = make_value(sun, 'rad', '.3f').loc[0]
sun_in.loc[r'$L\,(\si{\solarluminosity})$', 'Value'] = make_value(sun, 'lum').loc[0]
sun_in.loc[r'$\dnu\,(\si{\micro\hertz})$', 'Value'] = make_value(sun, 'dnu', '.1f').loc[0]
sun_in.loc[r'$\metallicity_\mathrm{surf}\,(\si{\dex})$', 'Value'] = make_value(sun, 'mhs').loc[0]

In [37]:
table = sun_in.reset_index()
table['Reference'] = references
table

,\textbf{Input},Value,Reference
0,"$M\,(\si{\solarmass})$",$1.000\pm0.001$,---
1,"$\tau\,(\si{\giga\year})$",$4.6\pm0.1$,\citet{Connelly.Bizzarro.ea2012}
2,"$\teff\,(\si{\kelvin})$",$5777\pm20$,\citet{Scott.Grevesse.ea2015}
3,"$R\,(\si{\solarradius})$",$1.000\pm0.001$,---
4,"$L\,(\si{\solarluminosity})$",$1.00\pm0.01$,---
5,"$\dnu\,(\si{\micro\hertz})$",$135.1\pm0.2$,\citet{Huber.Bedding.ea2011}
6,"$\metallicity_\mathrm{surf}\,(\si{\dex})$",$0.00\pm0.01$,\citet{Asplund.Grevesse.ea2009}


In [38]:
with open(f'../paper/sun_inputs.tex', 'w') as file:
    file.write(table.to_latex(escape=False, index=False, column_format='ccl'))

## Sun Results

In [2]:
sun_out = pd.read_csv('../modelling/final_models/sun_results/SUN/star/results.csv')
sun_out

,f_evol_16th,f_evol_50th,f_evol_84th,f_evol_ess_bulk,f_evol_ess_mean,f_evol_ess_sd,f_evol_ess_tail,f_evol_hpd_3%,f_evol_hpd_97%,f_evol_mcse_mean,...,mhs_ess_mean,mhs_ess_sd,mhs_ess_tail,mhs_hpd_3%,mhs_hpd_97%,mhs_mcse_mean,mhs_mcse_sd,mhs_mean,mhs_r_hat,mhs_sd
0,0.508369,0.516752,0.525287,6867.561222,6835.020599,6835.020599,8636.145873,0.500683,0.533182,0.000104,...,6560.933011,6560.933011,8366.3345,-0.018809,0.018475,0.000122,0.000086,0.000169,1.000672,0.009882


In [12]:
table = pd.DataFrame()
# table[r'\textbf{Name}'] = ['Sun']
table[r'$f_\mathrm{evol}$'] = make_result(sun_out, 'f_evol', '.3f')
table[r'$M\,(\si{\solarmass})$'] = make_result(sun_out, 'mass', '.3f')
if 'mlt_50th' in sun_out.columns:
    table[r'$\mlt$'] = make_result(sun_out, 'mlt', '.2f')
table[r'$Y_\mathrm{init}$'] = make_result(sun_out, 'yi', '.3f') 
table[r'$Z_\mathrm{init}$'] = make_result(sun_out, 'zi', '.4f')
# table[r'$\metallicity_\mathrm{init}\,(\si{\dex})$'] = make_result(sun_out, 'mhi', '.2f')    
table[r'$\tau\,(\si{\giga\year})$'] = make_result(sun_out, 'age', '.1f')
table[r'$\teff\,(\si{\kelvin})$'] = make_result(sun_out, 'teff', '.0f')
table[r'$R\,(\si{\solarradius})$'] = make_result(sun_out, 'rad', '.3f')
table[r'$\dnu\,(\si{\micro\hertz})$'] = make_result(sun_out, 'dnu', '.2f')
table[r'$\metallicity_\mathrm{surf}\,(\si{\dex})$'] = make_result(sun_out, 'mhs', '.2f')
table

,$f_\mathrm{evol}$,"$M\,(\si{\solarmass})$",$\mlt$,$Y_\mathrm{init}$,$Z_\mathrm{init}$,"$\tau\,(\si{\giga\year})$","$\teff\,(\si{\kelvin})$","$R\,(\si{\solarradius})$","$\dnu\,(\si{\micro\hertz})$","$\metallicity_\mathrm{surf}\,(\si{\dex})$"
0,$0.517\substack{+0.009 \\ -0.008}$,$1.000\substack{+0.001 \\ -0.001}$,$2.12\substack{+0.03 \\ -0.03}$,$0.262\substack{+0.002 \\ -0.002}$,$0.0150\substack{+0.0003 \\ -0.0003}$,$4.6\substack{+0.1 \\ -0.1}$,$5777\substack{+12 \\ -12}$,$1.001\substack{+0.001 \\ -0.001}$,$135.37\substack{+0.14 \\ -0.14}$,$0.00\substack{+0.01 \\ -0.01}$


In [13]:
# table = table.T
# table

In [14]:
# table.columns = [r'Value']
# table.index

In [15]:
# table.index.name = r'\textbf{Output}'

In [16]:
# table = table.reset_index()
# table

In [17]:
with open(f'../paper/sun_outputs.tex', 'w') as file:
    file.write(table.to_latex(escape=False, index=False, column_format='c'+'l'*(len(table.columns)-1)))

## ANN Validation

In [2]:
val = pd.read_csv('../training/validation.csv')
val

,\textbf{Error},$\mu_{1/2}$,$\sigma_\mathrm{MAD}$
0,"$\tau^\mathrm{true} - \tau^\mathrm{pred}\,(\ma...",-0.000080,0.006648
1,$T_\mathrm{eff}^\mathrm{true} - T_\mathrm{eff}...,-0.099828,1.594861
2,"$R^\mathrm{true} - R^\mathrm{pred}\,(\mathrm{R...",0.000026,0.000859
3,"$L^\mathrm{true} - L^\mathrm{pred}\,(\mathrm{L...",0.000830,0.002095
4,$\Delta\nu^\mathrm{true} - \Delta\nu^\mathrm{p...,-0.007138,0.083642
5,$[\mathrm{M}/\mathrm{H}]_\mathrm{surf}^\mathrm...,0.000028,0.000652


In [4]:
val[r'\textbf{Error}'] = [
    r'$\tau\,(\si{\giga\year})$',
    r'$T_\mathrm{eff}\,(\si{\kelvin})$',
    r'$R\,(\si{\solarmass})$',
    r'$L\,(\si{\solarluminosity})$',
    r'$\dnu\,(\si{\micro\hertz})$',
    r'$\metallicity_\mathrm{surf}\,(\si{\dex})$',
]

In [8]:
table = val.rename(columns={
    r'\textbf{Error}': r'\textbf{Output}'
})

In [9]:
with open(f'../paper/validation.tex', 'w') as file:
    file.write(table.to_latex(escape=False, index=False,
                              column_format='crr', float_format='{:.3f}'.format))

## Output CDS tables and README

In [34]:
tablemaker.writeCDSTables()

In [35]:
tablemaker.title = "Hierarchically modelling Kepler dwarfs and subgiants to improve inference " + \
    "of stellar properties with asteroseismology"
tablemaker.author = "Alexander J. Lyttle"
tablemaker.authors =           \
    "Guy R. Davies, " +        \
    "Tanda Li, " +             \
    "Lindsey M. Carboneau, " + \
    "Ho-Hin Leung, " +         \
    "Harry Westwood, " +       \
    "William J. Chaplin, " +   \
    "Oliver J. Hall, " +       \
    "Daniel Huber, " +         \
    "Martin B. Nielsen, " +    \
    "Sarbani Basu, " +         \
    "and Rafael A. García"
tablemaker.date = 2021

In [36]:
with open("../paper/main.tex") as file:
    tex = file.read()
    start = r'\begin{abstract}'
    end = r'\end{abstract}'
    ib = tex.find(start) + len(start)
    ie = tex.find(end)
    abstract = tex[ib:ie].strip()
print(abstract)

With recent advances in modelling stars using high-precision asteroseismology, the systematic effects associated with our assumptions of stellar helium abundance ($Y$) and the mixing-length theory parameter ($\alpha_{\rm MLT}$) are becoming more important. We apply a new method to improve the inference of stellar parameters for a sample of \emph{Kepler} dwarfs and subgiants across a narrow mass range ($0.8 < M < 1.2\,\rm M_\odot$). In this method, we include a statistical treatment of $Y$ and the $\alpha_{\rm MLT}$. We develop a hierarchical Bayesian model to encode information about the distribution of $Y$ and $\alpha_{\rm MLT}$ in the population, fitting a linear helium enrichment law including an intrinsic spread around this relation and normal distribution in $\alpha_{\rm MLT}$. We test various levels of pooling parameters, with and without solar data as a calibrator. When including the Sun as a star, we find the gradient for the enrichment law, $\Delta Y / \Delta Z = 1.05\substack

In [37]:
tablemaker.abstract = abstract

In [38]:
tablemaker.bibcode = 'bibcode'

In [39]:
description = "Tables with the observed data and posterior summary for the fundamental parameters of " + \
    "a sample of dwarf and subgiant stars. All stars are from the Kepler Input Catalog (KIC) except for " + \
    "the PPS and MPS models which include the sun as a star (SUN)."

In [40]:
tablemaker.setReadmeTemplate(
    'ReadMe.template', 
    {'description': tablemaker._CDSTablesMaker__splitLine(description, shift=2)})

In [41]:
for table in tablemaker.getTables():
    table.setByteByByteTemplate('bytebybyte.template')

In [42]:
# Save ReadMe into a file
with open("ReadMe", "w") as fd:
    tablemaker.makeReadMe(out=fd)

Some manual edits are needed.